In [1]:
%store -r s3_bucket
%store -r prefix
%store -r training_job_name

In [2]:
path = 's3://{}/{}/input/{}'.format(
    s3_bucket, 
    prefix, 
    "synthetic.test_without_labels.txt"
)
s3_test_without_labels_data = path

path = 's3://{}/{}/batch-prediction'.format(
    s3_bucket, 
    prefix
)
batch_output = path

In [3]:
from sagemaker.estimator import Estimator

estimator = Estimator.attach(training_job_name)


2021-06-01 10:13:37 Starting - Preparing the instances for training
2021-06-01 10:13:37 Downloading - Downloading input data
2021-06-01 10:13:37 Training - Training image download completed. Training in progress.
2021-06-01 10:13:37 Uploading - Uploading generated training model
2021-06-01 10:13:37 Completed - Training job completed


In [4]:
transformer = estimator.transformer(
    instance_count=1, 
    instance_type='ml.m4.xlarge', 
    output_path=batch_output
)

In [5]:
transformer.transform(
    data=s3_test_without_labels_data, 
    data_type='S3Prefix',
    content_type='application/jsonlines', 
    split_type='Line'
)

transformer.wait()

...............................Arguments: serve
Arguments: serve
[06/01/2021 10:40:16 INFO 140349593466240] Finding and loading model
[06/01/2021 10:40:16 INFO 140349593466240] Trying to load model from /opt/ml/model/model.bin
[06/01/2021 10:40:17 INFO 140349593466240] Number of server workers: 4
[2021-06-01 10:40:17 +0000] [1] [INFO] Starting gunicorn 19.7.1
[2021-06-01 10:40:17 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2021-06-01 10:40:17 +0000] [1] [INFO] Using worker: sync
[2021-06-01 10:40:17 +0000] [32] [INFO] Booting worker with pid: 32
[2021-06-01 10:40:17 +0000] [33] [INFO] Booting worker with pid: 33
[2021-06-01 10:40:17 +0000] [34] [INFO] Booting worker with pid: 34
[2021-06-01 10:40:17 +0000] [35] [INFO] Booting worker with pid: 35
[06/01/2021 10:40:16 INFO 140349593466240] Finding and loading model
[06/01/2021 10:40:16 INFO 140349593466240] Trying to load model from /opt/ml/model/model.bin
[06/01/2021 10:40:17 INFO 140349593466240] Number of server workers: 

In [6]:
!aws s3 cp {batch_output} ./tmp --recursive

download: s3://sagemaker-cookbook-bucket/chapter08/batch-prediction/synthetic.test_without_labels.txt.out to tmp/synthetic.test_without_labels.txt.out


In [7]:
!head tmp/synthetic.test_without_labels.txt.out

{"label": ["__label__negative"], "prob": [0.609320878982544]}
{"label": ["__label__negative"], "prob": [0.6484045386314392]}
{"label": ["__label__positive"], "prob": [0.5025691986083984]}
{"label": ["__label__negative"], "prob": [0.7442595958709717]}
{"label": ["__label__negative"], "prob": [0.6708104610443115]}
{"label": ["__label__negative"], "prob": [0.5628624558448792]}
{"label": ["__label__negative"], "prob": [0.6374379992485046]}
{"label": ["__label__negative"], "prob": [0.7534217238426208]}
{"label": ["__label__positive"], "prob": [0.8219373226165771]}
{"label": ["__label__positive"], "prob": [0.9115214943885803]}


In [8]:
!head tmp/synthetic.test_without_labels.txt

 {"source":  "There  Are  Better  Restaurants  Out  There  tastes  bad  spaghetti  chicken  soup  food  in  the  restaurant"} 
 {"source":  "Not  Impressive  i  will  not  recommend  this  to  my  friends  there  are  better  restaurants  out  there  spaghetti  chicken  soup  food  in  the  restaurant  donut"} 
 {"source":  "I  Will  Not  Recommend  This  To  My  Friends  spaghetti  chicken  soup  food  in  the  restaurant"} 
 {"source":  "There  Are  Better  Restaurants  Out  There  there  are  better  restaurants  out  there  spaghetti  chicken  soup  tastes  bad  dinner  time  tastes  bad  this  is  bad"} 
 {"source":  "Spaghetti  Chicken  Soup  this  is  bad  i  will  not  recommend  this  to  my  friends  i  hate  it  very  bad"} 
 {"source":  "Not  Impressive  spaghetti  chicken  soup  i  will  not  recommend  this  to  my  friends  spaghetti  chicken  soup"} 
 {"source":  "There  Are  Better  Restaurants  Out  There  spaghetti  chicken  soup  i  hate  it  this  is  bad  i  hate 